
# Gram–Schmidt Programming Exercise (Completed & Organized)

This notebook demonstrates:

- Using NumPy's linear algebra tools (`numpy.linalg`)  
- Writing robust helper functions with `try` / `except`  
- Implementing:
  - a `normalize` function for vectors  
  - a `project` function (projection of one vector onto another)  
  - a full `GramSchmidt` orthonormalization algorithm  
- Testing the implementation on random examples


## 1. Setup & Imports

In [1]:
import numpy as np
from numpy import linalg as la

# Random number generator (for tests)
drg = np.random.default_rng()

## 2. Norm Demo

We use the Euclidean norm:

\\[
\|v\| = \sqrt{v_0^2 + v_1^2 + \cdots + v_{n-1}^2}.
\\]

NumPy provides this as `numpy.linalg.norm`.


In [2]:
# Play around with this to be sure you understand it.
v = np.array([1, 2, 3])
print("v =", v)
print("la.norm(v) =", la.norm(v))
print("Manual sqrt(1^2 + 2^2 + 3^2) =", np.sqrt(1**2 + 2**2 + 3**2))
print("Equality check:", la.norm(v) == np.sqrt(1**2 + 2**2 + 3**2))

v = [1 2 3]
la.norm(v) = 3.7416573867739413
Manual sqrt(1^2 + 2^2 + 3^2) = 3.7416573867739413
Equality check: True


## 3. Normalizing Function `normalize`

We want a function that:

1. Takes a vector `v` and returns the normalized version

   \\[
   \frac{v}{\|v\|}.
   \\]

2. If the norm of `v` is zero, or `v` is otherwise invalid, it should:

   - Print a clear error message  
   - Return the original vector `v` unchanged


In [3]:
# returns a normalized version of v
def normalize(v):
    """Return the normalized version of v (v / ||v||).

    If ||v|| == 0 or another error occurs, print an error message
    and return the original vector v unchanged.
    """
    try:
        # Check if the norm of v is zero
        if la.norm(v) == 0:
            raise ZeroDivisionError
        else:
            return v / la.norm(v)
    except ZeroDivisionError:
        # Give an appropriate error message if the vector failed its norm check
        print("Error: cannot normalize the zero vector (norm is 0). Returning original vector.")
        return v
    except Exception as S:
        # General failure: print message and return original vector
        print("Error in normalize(v):", S)
        return v


# Quick tests for normalize
print("normalize([1, 2, 3]) =", normalize(np.array([1, 2, 3])))
print("normalize([0, 0, 0]) =", normalize(np.array([0, 0, 0])))

normalize([1, 2, 3]) = [0.26726124 0.53452248 0.80178373]
Error: cannot normalize the zero vector (norm is 0). Returning original vector.
normalize([0, 0, 0]) = [0 0 0]


## 4. Projection Function `project`

Given vectors $u$ and $v$, the projection of $u$ onto $v$ is:

\\[
\operatorname{proj}_v u = \frac{u \cdot v}{v \cdot v} \, v.
\\]

Our function should:

- Accept two vectors `u` and `v`  
- Return the projection of `u` onto `v`  
- If `v` has zero norm or another error occurs, print a helpful message and return `v` unchanged


In [4]:
# projects u onto v
def project(u, v):
    """Project u onto v.

    Returns (u·v / v·v) * v, unless ||v|| == 0 or an error occurs,
    in which case it prints an error and returns v unchanged.
    """
    try:
        # Check if the norm of v is zero
        if la.norm(v) == 0:
            raise ZeroDivisionError
        else:
            return (u @ v) / (v @ v) * v
    except ZeroDivisionError:
        print("Error: cannot project onto the zero vector. Returning v unchanged.")
        return v
    except Exception as S:
        print("Error in project(u, v):", S)
        return v


# Quick tests for project
u_test = np.array([1., 2., 3.])
v_test = np.array([4., 0., 0.])
print("project(u_test, v_test) =", project(u_test, v_test))

v_zero = np.array([0., 0., 0.])
print("project(u_test, v_zero) =", project(u_test, v_zero))

project(u_test, v_test) = [1. 0. 0.]
Error: cannot project onto the zero vector. Returning v unchanged.
project(u_test, v_zero) = [0. 0. 0.]


## 5. Gram–Schmidt Orthonormalization

We now implement the Gram–Schmidt process.

Given a list of vectors `B = [b₁, b₂, ..., bₖ]`, we want to construct an **orthonormal basis**
`New_B = [q₁, q₂, ..., qₘ]` that spans the same subspace.

Algorithm outline:

1. Initialize an empty list `New_B`.
2. For each vector `b` in `B`:
   - Start with `new_vec` as the zero vector.
   - Subtract off all projections of `b` onto the vectors in `New_B`:
     \\[
     \text{new\_vec} = b - \sum_{c \in \text{New\_B}} \operatorname{proj}_c b.
     \\]
   - If `new_vec` is not (numerically) the zero vector, normalize it and append to `New_B`.
3. Return the resulting orthonormal vectors as the columns of a NumPy array.


In [5]:
# applies the Gram–Schmidt orthogonalization algorithm with normalization
def GramSchmidt(B):
    """Apply Gram–Schmidt to a list of vectors B.

    Parameters
    ----------
    B : list-like of 1D NumPy arrays
        Input vectors.

    Returns
    -------
    np.ndarray
        A matrix whose columns are an orthonormal basis spanning the same
        subspace as the original B.

    On error, prints a message and returns the original list B as-is.
    """
    try:
        # Create a new empty list for the orthonormal basis
        New_B = []
        for b in B:
            # Create a zero vector of the same length as b
            new_vec = np.zeros(len(b))

            # Subtract the projections of b onto each existing basis vector in New_B
            for c in New_B:
                new_vec = new_vec + project(b, c)

            # Replace with the part of b orthogonal to all vectors in New_B
            new_vec = b - new_vec

            # If the norm of new_vec isn't zero, normalize it and add it to New_B
            if la.norm(new_vec) != 0:
                new_vec = normalize(new_vec)
                New_B.append(new_vec)

    except Exception as S:
        print("Error in GramSchmidt(B):", S)
        return B

    # Return New_B as a NumPy array with each vector as a column (orthonormal matrix)
    return np.array(New_B).T


# Simple sanity check in low dimension
B_example = [np.array([1., 1., 0.]),
             np.array([1., 0., 1.]),
             np.array([0., 1., 1.])]

Q_example = GramSchmidt(B_example)
print("Q_example =\n", Q_example)
print("Q_example^T Q_example =\n", Q_example.T @ Q_example)

Q_example =
 [[ 0.70710678  0.40824829 -0.57735027]
 [ 0.70710678 -0.40824829  0.57735027]
 [ 0.          0.81649658  0.57735027]]
Q_example^T Q_example =
 [[1.00000000e+00 2.45142679e-17 2.73204007e-17]
 [2.45142679e-17 1.00000000e+00 9.80784505e-17]
 [2.73204007e-17 9.80784505e-17 1.00000000e+00]]


## 6. Testing `GramSchmidt`

We now define a helper function `test_gs` that:

- Applies `GramSchmidt` to a given list of vectors `B`  
- Checks whether the resulting matrix `Q` is orthonormal by testing if

  \\[ Q^T Q \approx I \]

  using `numpy.testing.assert_allclose`


In [6]:
from numpy.testing import assert_allclose

def test_gs(B):
    """Test GramSchmidt on list of vectors B.

    Returns True if Q^T Q is close to the identity, otherwise False.
    """
    success = True
    Bprime = GramSchmidt(B)
    test_matrix = Bprime @ Bprime.T
    ident = np.eye(test_matrix.shape[0])
    try:
        assert_allclose(test_matrix, ident, atol=1e-10)
    except AssertionError:
        success = False
        print("Assertion that values are close failed")
    except Exception as S:
        success = False
        print("Test failed for other reasons:\n", S)
    return success


# Quick test in 3D
B_test = [np.array([1., 2., 3.]),
          np.array([4., 5., 6.]),
          np.array([7., 8., 10.])]
print("Single test_gs(B_test) ->", test_gs(B_test))

Single test_gs(B_test) -> True


## 7. Randomized Stress Tests

Finally, we test our Gram–Schmidt implementation on random integer vectors
of varying dimension.


In [7]:
for i in range(10):
    dimension = drg.integers(low=3, high=200)
    # Create a list of 'dimension' random integer vectors of length 'dimension'
    B = [drg.integers(low=0, high=10, size=dimension).astype(float)
         for _ in range(dimension)]
    result = test_gs(B)
    if result:
        print("Test %i: For dimension %i we have success!" % (i + 1, dimension))
    else:
        print("Test %i: For dimension %i we had a failure!" % (i + 1, dimension))

Test 1: For dimension 131 we have success!
Test 2: For dimension 40 we have success!
Test 3: For dimension 172 we have success!
Test 4: For dimension 54 we have success!
Test 5: For dimension 102 we have success!
Test 6: For dimension 125 we have success!
Test 7: For dimension 173 we have success!
Test 8: For dimension 101 we have success!
Test 9: For dimension 69 we have success!
Test 10: For dimension 24 we have success!



### Summary

In this notebook we:

- Used NumPy's `norm` to understand vector length  
- Implemented robust helper functions `normalize` and `project` with error handling  
- Built a full Gram–Schmidt orthonormalization routine `GramSchmidt`  
- Verified correctness numerically using orthonormality tests on random data

This gives both practice with **linear algebra** and with **defensive programming** in Python.
